In [143]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_models
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
import numpy as np

In [144]:
from functions import naive_ensemble

In [145]:
daxdata = get_DAX.get()

In [146]:
base = DAX_baseline.DAX_baseline(daxdata)

In [147]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.957440,-0.510195,0.073395,0.715653,2.719772
1,2024-02-07,DAX,2 day,-3.936562,-0.759111,0.100316,0.978977,3.570492
2,2024-02-07,DAX,5 day,-4.615241,-0.853908,0.168228,1.251251,4.628697
3,2024-02-07,DAX,6 day,-5.246158,-0.921253,0.218176,1.451656,5.367396
4,2024-02-07,DAX,7 day,-5.683390,-0.970269,0.344808,1.607134,6.259934


In [148]:
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [149]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.957440,-0.510195,0.073395,0.715653,2.719772
1,2024-02-07,DAX,2 day,-3.936562,-0.759111,0.100316,0.978977,3.570492
2,2024-02-07,DAX,5 day,-4.615241,-0.853908,0.168228,1.251251,4.628697
3,2024-02-07,DAX,6 day,-5.246158,-0.921253,0.218176,1.451656,5.367396
4,2024-02-07,DAX,7 day,-5.683390,-0.970269,0.344808,1.607134,6.259934


In [150]:
quantile

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.209547,-0.493443,0.002910,0.498812,1.640074
1,2024-02-07,DAX,2 day,-0.885366,0.836554,1.456188,1.955771,3.507276
2,2024-02-07,DAX,5 day,-1.161187,0.593248,1.236310,1.702845,3.029730
3,2024-02-07,DAX,6 day,-0.619690,1.138833,1.863729,2.386058,3.608379
4,2024-02-07,DAX,7 day,-1.036068,0.598991,1.264714,1.688108,3.057734


In [151]:
df_dax = naive_ensemble.ensemble(base,quantile,0.8,0.2)

In [152]:
df_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.807862,-0.506845,0.059298,0.672285,2.503832
1,2024-02-07,DAX,2 day,-3.326323,-0.439978,0.371490,1.174336,3.557849
2,2024-02-07,DAX,5 day,-3.924430,-0.564477,0.381845,1.341569,4.308904
3,2024-02-07,DAX,6 day,-4.320864,-0.509236,0.547286,1.638536,5.015593
4,2024-02-07,DAX,7 day,-4.753926,-0.656417,0.528789,1.623329,5.619494


In [153]:
energydata= get_energy.get()

  0%|          | 0/268 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 268/268 [00:27<00:00,  9.74it/s]


In [154]:
df_energy= energy_models.energy_XGBoost(energydata,date_str='2024-02-07')

In [155]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,energy,36 hour,64.022408,69.107910,72.117012,74.818108,76.808556
1,2024-02-07,energy,40 hour,60.963951,64.634949,67.998154,69.369057,72.419327
2,2024-02-07,energy,44 hour,59.942081,62.896214,64.971146,67.447548,68.162323
3,2024-02-07,energy,60 hour,57.347794,59.956478,61.959045,63.195324,65.075684
4,2024-02-07,energy,64 hour,53.532665,56.130779,57.538334,58.280888,62.741554
5,2024-02-07,energy,68 hour,53.326565,57.302132,58.636341,59.574402,60.950912


In [156]:
#Reorder quantiles to prevent quantile crossing

In [157]:
df = merge_submissions.merge(df_energy,df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2024-02-07         DAX    1 day  -2.807862  -0.506845   0.059298   
1    2024-02-07         DAX    2 day  -3.326323  -0.439978    0.37149   
2    2024-02-07         DAX    5 day   -3.92443  -0.564477   0.381845   
3    2024-02-07         DAX    6 day  -4.320864  -0.509236   0.547286   
4    2024-02-07         DAX    7 day  -4.753926  -0.656417   0.528789   
0    2024-02-07      energy  36 hour  64.022408   69.10791  72.117012   
1    2024-02-07      energy  40 hour  60.963951  64.634949  67.998154   
2    2024-02-07      energy  44 hour  59.942081  62.896214  64.971146   
3    2024-02-07      energy  60 hour  57.347794  59.956478  61.959045   
4    2024-02-07      energy  64 hour  53.532665  56.130779  57.538334   
5    2024-02-07      energy  68 hour  53.326565  57.302132  58.636341   
0    2024-02-07  infections   0 week         NA         NA         NA   
1    2024-02-07  infections   1 week         NA    

In [158]:
import pandas as pd

In [159]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [160]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.807862,-0.506845,0.059298,0.672285,2.503832
1,2024-02-07,DAX,2 day,-3.326323,-0.439978,0.37149,1.174336,3.557849
2,2024-02-07,DAX,5 day,-3.92443,-0.564477,0.381845,1.341569,4.308904
3,2024-02-07,DAX,6 day,-4.320864,-0.509236,0.547286,1.638536,5.015593
4,2024-02-07,DAX,7 day,-4.753926,-0.656417,0.528789,1.623329,5.619494
0,2024-02-07,energy,36 hour,64.022408,69.10791,72.117012,74.818108,76.808556
1,2024-02-07,energy,40 hour,60.963951,64.634949,67.998154,69.369057,72.419327
2,2024-02-07,energy,44 hour,59.942081,62.896214,64.971146,67.447548,68.162323
3,2024-02-07,energy,60 hour,57.347794,59.956478,61.959045,63.195324,65.075684
4,2024-02-07,energy,64 hour,53.532665,56.130779,57.538334,58.280888,62.741554


In [161]:
df.to_csv('../../forecasts/20240207_JonSnow.csv',index=False)

In [162]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-07,DAX,1 day,-2.807862,-0.506845,0.059298,0.672285,2.503832
1,2024-02-07,DAX,2 day,-3.326323,-0.439978,0.37149,1.174336,3.557849
2,2024-02-07,DAX,5 day,-3.92443,-0.564477,0.381845,1.341569,4.308904
3,2024-02-07,DAX,6 day,-4.320864,-0.509236,0.547286,1.638536,5.015593
4,2024-02-07,DAX,7 day,-4.753926,-0.656417,0.528789,1.623329,5.619494
0,2024-02-07,energy,36 hour,64.022408,69.10791,72.117012,74.818108,76.808556
1,2024-02-07,energy,40 hour,60.963951,64.634949,67.998154,69.369057,72.419327
2,2024-02-07,energy,44 hour,59.942081,62.896214,64.971146,67.447548,68.162323
3,2024-02-07,energy,60 hour,57.347794,59.956478,61.959045,63.195324,65.075684
4,2024-02-07,energy,64 hour,53.532665,56.130779,57.538334,58.280888,62.741554


In [163]:
from datetime import timedelta, datetime

In [164]:
from functions import check_submission

In [165]:
check_submission.check_df(df)

Start checking...
---------------------------
Checking the Columns...
Checking type of columns...
----WARNING: Some elements in q0.025 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.25 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.5 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.75 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.975 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

Checking if the Dates make sense...
Checking targets...
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA values.
Skipping plausibility check for 'infections' as it contains NA values.
Checking quantiles...
Seems like you submitted 